Вот описание двух финальных скриптов, которые ты используешь в пайплайне PhotoMaps:

🛠️ Скрипт 1: Сжатие изображений и генерация CSV (compress_and_index.py)

Задача:
Обрабатывает файлы из images/:
	•	Извлекает координаты и дату (EXIF + JSON)
	•	Если координаты есть — сохраняет сжатую копию в photos/
	•	Переименовывает файл по шаблону IMG_YYYYMMDD_hash.jpg, если дата не указана в имени
	•	Добавляет информацию (имя, координаты, дата) в csv/photos.csv
	•	Автоматически разбивает photos.csv на файлы по годам: photos_2020.csv, photos_2021.csv и т.д.
	•	Все отбраковки (без координат или ошибок) — в csv/errors_combined.csv

Вход:
	•	images/*.jpg|.jpeg
	•	(опционально) .json рядом с фото (.supplemental-metadata.json)

Выход:
	•	photos/*.jpg — сжатые изображения с гарантированными координатами
	•	csv/photos.csv — полный набор метаданных
	•	csv/photos_YYYY.csv — разбитые по годам
	•	csv/errors_combined.csv — лог ошибок

🎨 Скрипт 2: Генерация миниатюр (generate_thumbnails.py)

Задача:
Создает круглые миниатюры 64x64 PNG для каждого сжатого фото из photos/, окрашивает рамку по году.

Особенности:
	•	Превью оформлено как круг с цветной рамкой
	•	Цвет зависит от года (из палитры)
	•	Названия .jpg/.jpeg → .png

Вход:
	•	photos/*.jpg
	•	csv/photos.csv (для определения года)

Выход:
	•	thumbnails/*.png — иконки, отображаемые на карте

🔄 Обновлённый пайплайн
	1.	images/ — исходные JPG-файлы (и .json при наличии)
	2.	▶️ Скрипт 1: compress_and_index.py
	•	➡️ photos/ (JPEG, компрессия)
	•	➡️ csv/ (метаданные)
	3.	▶️ Скрипт 2: generate_thumbnails.py
	•	➡️ thumbnails/ (иконки PNG)
	4.	🗺️ Карта отображает точки с превью и попапами, данные из csv/photos_*.csv


In [ ]:
Пайплайн
=======

0. ВСЕ фото положить в папку images/

PhotoMaps.ipynb - 
1. script to study our images. Retrieves how many files of which types are there in source folder images/--

OUTPUT
📁 Анализ папки: images
===================================
  .JPG: 2185
 .JPEG: 20
 .HEIC: 6
 .JSON: 2068
🌀 Дубликатов с (1), (2)...: 5
📋 Дубликатов copy         : 4
✂️  -edited файлов         : 146
===================================
📦 Всего файлов: 4282

2. # -- duplicate images eliminator -- run every time you download the new copy of google photos
# -- file type: IMG_1234(1).JPG, IMG_1234(2).JPG and their json (if any)

OUTPUT
🔍 Найдены неправильные JSON файлы:
 - EFFECTS.jpg.supplemental-metadata(1).json
 - IMG_2822.JPG.supplemental-metadata(1).json
 - IMG_0128.JPG.supplemental-metadata(1).json

3. # -- duplicate images eliminator -- run every time you download the new copy of google photos
# -- file type: IMG_1234 copy.JPG, IMG_1234 copy copy.JPG and their json (if any)

=== 📋 РЕЗУЛЬТАТЫ ОБРАБОТКИ ===
🔍 Найдено файлов-копий: 4
📌 Файлов, содержащих EXIF: 0
✅ Успешно обработано JSON файлов: 4
❌ JSON не найден для битмапов: 0

4. # -- scripts to check HEIC files  -- 
🔍 Проверка .HEIC в папке images
Найдено: 6 HEIC файлов
❌ Без 'ftyp': 6
 - IMG_1894.HEIC
 - IMG_1899.HEIC

5. # -- HEIC > JPG converter
✅ Конвертировано и удалено: 6 HEIC → JPG

6. # --- images>photos converter  - MAIN script --
✅ С координатами: 2065 → /Users/mloktionov/PycharmProjects/PhotoMaps/csv/photos.csv
⚠️ Без координат : 0 → /Users/mloktionov/PycharmProjects/PhotoMaps/csv/missing_coords.csv
📁 Фото сохранены в: /Users/mloktionov/PycharmProjects/PhotoMaps/photos
📝 Лог записан в: /Users/mloktionov/PycharmProjects/PhotoMaps/csv/import_log.txt
🌍 Джиттер применён для повторяющихся координат

✅ Завершено. Результаты в photos.csv и missing_coords.csv

7. # ------ Generating preview thumbnails in circles of a certain color (now OBSOLETE)

(уже не нужен)

✅ Превью создано: 2065
⚠️ Пропущено: 0
📁 Сохранено в: /Users/mloktionov/PycharmProjects/PhotoMaps/thumbnails

8. # ---- Getting descriptions from Google photos via API

🔍 Поиск альбомов в Google Photos...
✅ Найдены альбомы: ['PhotoMap 2022-2025', 'PhotoMap 2019-2021', 'PhotoMap 2017-2019', 'PhotoMap 2014-2016']

📂 Альбом: PhotoMap 2022-2025
Загрузка из PhotoMap 2022-2025: 653фото [00:07, 89.61фото/s] 

📂 Альбом: PhotoMap 2019-2021
Загрузка из PhotoMap 2019-2021: 784фото [00:07, 100.08фото/s]

📂 Альбом: PhotoMap 2017-2019
Загрузка из PhotoMap 2017-2019: 249фото [00:02, 95.38фото/s] 

📂 Альбом: PhotoMap 2014-2016
Загрузка из PhotoMap 2014-2016: 381фото [00:05, 75.50фото/s]

✅ Все данные успешно записаны в csv/photos_descriptions.csv

In [ ]:
# -- script to study our images. Retrieves how many files of which types are there in source folder images/--

In [ ]:
import os
import re
from collections import Counter

FOLDER = "images"  # замени на абсолютный путь при необходимости
files = os.listdir(FOLDER)

# === Расширения
ext_counts = Counter()
suffix_counts = Counter()
edited_count = 0
copy_count = 0

for f in files:
    lower = f.lower()
    if lower.endswith(('.jpg', '.jpeg', '.heic')):
        ext = os.path.splitext(f)[1].lower()
        ext_counts[ext] += 1

        # дубликаты: IMG_1234(1).JPG
        if re.search(r'\(\d+\)\.(jpe?g|heic)$', lower):
            suffix_counts['duplicate'] += 1

        # -edited.JPG
        if '-edited' in lower:
            edited_count += 1

        # " copy" в имени файла
        if re.search(r' copy(\s\d+)?\.(jpe?g|heic)$', lower):
            copy_count += 1

    elif lower.endswith('.json'):
        ext_counts['.json'] += 1

# === Вывод
print(f"📁 Анализ папки: {FOLDER}")
print("===================================")
for ext in ['.jpg', '.jpeg', '.heic', '.json']:
    print(f"{ext.upper():>6}: {ext_counts[ext]}")

print(f"🌀 Дубликатов с (1), (2)...: {suffix_counts['duplicate']}")
print(f"📋 Дубликатов copy         : {copy_count}")
print(f"✂️  -edited файлов         : {edited_count}")
print("===================================")
print(f"📦 Всего файлов: {len(files)}")

In [ ]:
# -- duplicate images eliminator -- run every time you download the new copy of google photos
# -- file type: IMG_1234(1).JPG, IMG_1234(2).JPG and their json (if any)

In [ ]:
import os
import re
import json

# === 📁 Папка с изображениями
folder = "images"

# === 🎯 Регулярка для поиска неправильных JSON
pattern = re.compile(r"(.*)\.supplemental-metadata\((\d+)\)\.json", re.IGNORECASE)

# === 🔄 Поиск всех JSON с неправильным именем
json_files = [f for f in os.listdir(folder) if pattern.match(f)]

# === 🔎 Логируем найденные файлы
if json_files:
    print(f"🔍 Найдены неправильные JSON файлы:")
    for f in json_files:
        print(f" - {f}")
else:
    print("⚠️ Не найдено файлов для переименования!")

for fname in json_files:
    json_path = os.path.join(folder, fname)
    
    # === ✅ Проверяем, что файл существует
    if not os.path.exists(json_path):
        print(f"❌ Ошибка: файл {json_path} не найден перед переименованием!")
        continue

    print(f"🔄 Обработка файла: {json_path}")
    
    # === 🔍 Получаем данные из регулярного выражения
    match = pattern.match(fname)
    base_name, index = match.groups()
    print(f"   → Базовое имя: {base_name}, Индекс: {index}")
    
    # === 📝 Чистим название от расширений, чтобы не дублировались
    if base_name.lower().endswith('.jpg'):
        base_name = base_name[:-4]
    elif base_name.lower().endswith('.jpeg'):
        base_name = base_name[:-5]

    # === 📝 Формируем правильное имя
    correct_name = f"{base_name}({index}).JPG.supplemental-metadata.json"
    correct_path = os.path.join(folder, correct_name)
    
    print(f"🔄 Переименование: {json_path} → {correct_path}")
    
    # === 🔄 Переименование
    try:
        os.rename(json_path, correct_path)
        
        # === ✅ Проверяем, что файл появился после переименования
        if not os.path.exists(correct_path):
            print(f"❌ Ошибка: файл {correct_path} не появился после переименования!")
            continue
        
    except Exception as e:
        print(f"❌ Ошибка при переименовании: {e}")
        continue
    
    # === 🔄 Читаем JSON и исправляем title
    try:
        with open(correct_path, "r", encoding="utf-8") as file:
            metadata = json.load(file)
        
        # === Новое значение для title
        old_title = metadata.get("title", "")
        new_title = f"{base_name}({index}).JPG"
        
        if old_title != new_title:
            print(f"🔄 Исправление title: {old_title} → {new_title}")
            metadata["title"] = new_title
            
            # === 📝 Записываем обратно
            with open(correct_path, "w", encoding="utf-8") as file:
                json.dump(metadata, file, ensure_ascii=False, indent=4)
        else:
            print(f"✅ Title уже корректный: {old_title}")
    
    except Exception as e:
        print(f"❌ Ошибка при обработке JSON: {correct_path} → {e}")

In [ ]:
# -- duplicate images eliminator -- run every time you download the new copy of google photos
# -- file type: IMG_1234 copy.JPG, IMG_1234 copy copy.JPG and their json (if any)

In [ ]:
import os
import re
import json
from PIL import Image, ExifTags

# === 📁 Папка с изображениями
folder = "images"

# === 🗂️ Проверяем, существует ли папка
if not os.path.exists(folder):
    print(f"❌ Папка '{folder}' не найдена. Создаю её...")
    os.makedirs(folder)
    print(f"✅ Папка '{folder}' создана. Помести в неё файлы для обработки.")
    exit()

# === 👁️ Проверка на наличие GPS в EXIF
def has_gps(exif):
    gps_tag_id = [k for k, v in ExifTags.TAGS.items() if v == "GPSInfo"]
    if not gps_tag_id:
        return False
    gps_data = exif.get(gps_tag_id[0])
    return bool(gps_data)

# === 🎯 Регулярное выражение для поиска цепочек " copy"
pattern = re.compile(r"^(.+?)( copy)+(\s\d+)?(\..+)$", re.IGNORECASE)
candidates = [f for f in os.listdir(folder) if pattern.search(f) and f.lower().endswith(('.jpg', '.jpeg', '.png', '.heic'))]

# === 🗂️ Списки результатов
not_found_json = []
updated_json_files = []
missing_gps_and_json = []
files_with_gps = []

print(f"🔎 Найдено файлов с меткой ' copy': {len(candidates)}")

for fname in candidates:
    base, ext = os.path.splitext(fname)
    original_base = base.replace(' copy', '')
    
    # === 🔄 Формируем имена JSON для всех регистров
    json_candidates = [
        f"{original_base}.jpg.supplemental-metadata copy.json",
        f"{original_base}.jpeg.supplemental-metadata copy.json",
        f"{original_base}.JPG.supplemental-metadata copy.json",
        f"{original_base}.JPEG.supplemental-metadata copy.json",
    ]
    
    # === 🔄 Ищем существующие JSON
    existing_jsons = [j for j in json_candidates if os.path.exists(os.path.join(folder, j))]
    
    if existing_jsons:
        json_path = os.path.join(folder, existing_jsons[0])
        print(f"\n✅ JSON найден для: {fname} → {existing_jsons[0]}")

        # === 📝 Переименовываем в правильный формат
        new_json_name = f"{fname}.supplemental-metadata.json"
        new_json_path = os.path.join(folder, new_json_name)
        os.rename(json_path, new_json_path)

        # === 🔄 Обновляем "title"
        with open(new_json_path, "r", encoding="utf-8") as file:
            metadata = json.load(file)

        original_title = metadata.get("title", "")
        if original_title != fname:
            print(f"🔄 Обновление 'title' в JSON: {original_title} → {fname}")
            metadata["title"] = fname
        
            # === 📝 Сохраняем обратно
            with open(new_json_path, "w", encoding="utf-8") as file:
                json.dump(metadata, file, ensure_ascii=False, indent=4)

        updated_json_files.append(fname)
        print(f"✅ JSON для {fname} успешно обработан → {new_json_path}")
    else:
        print(f"\n❌ JSON не найден для: {fname}")
        not_found_json.append(fname)
        continue

    # === ✅ 1. Если есть EXIF — пропускаем
    img_path = os.path.join(folder, fname)
    try:
        img = Image.open(img_path)
        exif = img._getexif() or {}
        if has_gps(exif):
            files_with_gps.append(fname)
            print(f"🌍 EXIF-координаты найдены в {fname}, JSON не требуется.")
            continue
    except Exception as e:
        missing_gps_and_json.append(f"{fname} (ошибка чтения EXIF: {e})")
        print(f"❌ Ошибка чтения EXIF: {fname} → {e}")
        continue

# === 📝 Итоги
print("\n=== 📋 РЕЗУЛЬТАТЫ ОБРАБОТКИ ===")
print(f"🔍 Найдено файлов-копий: {len(candidates)}")
print(f"📌 Файлов, содержащих EXIF: {len(files_with_gps)}")
print(f"✅ Успешно обработано JSON файлов: {len(updated_json_files)}")
print(f"❌ JSON не найден для битмапов: {len(not_found_json)}")

# === 📂 Полный список всех найденных файлов-копий
print("\n📂 Полный список всех найденных файлов-копий:")
for f in candidates:
    print(f" - {f}")

if not_found_json:
    print("\n⛔ Список не найденных JSON для изображений:")
    for f in not_found_json:
        print(f" - {f}")

In [ ]:
# -- scripts to convert HEIC files into jpgs -- 

In [ ]:
import os

# 📁 Укажи путь к папке с изображениями
folder = "images"

# 🔍 Собираем все .HEIC файлы
heic_files = [f for f in os.listdir(folder) if f.lower().endswith(".heic")]

invalid_files = []

for fname in heic_files:
    path = os.path.join(folder, fname)
    try:
        with open(path, "rb") as f:
            header = f.read(512)
            if b"ftyp" not in header:
                invalid_files.append(fname)
    except Exception as e:
        invalid_files.append(f"{fname} (ошибка чтения: {e})")

# 📋 Выводим список подозрительных файлов
print(f"\n🔍 Проверка .HEIC в папке {folder}")
print(f"Найдено: {len(heic_files)} HEIC файлов")
print(f"❌ Без 'ftyp': {len(invalid_files)}")
for f in invalid_files:
    print(f" - {f}")

In [ ]:
# -- HEIC > JPG converter

In [ ]:
import os
import subprocess
import json

# === 📁 Папка с изображениями
folder = "images"

# === 🔍 Находим все HEIC файлы
heic_files = [f for f in os.listdir(folder) if f.lower().endswith(".heic")]
converted = []
failed = []

# === 🔄 Конвертация HEIC → JPG и переименование JSON
for fname in heic_files:
    in_path = os.path.join(folder, fname)
    out_name = os.path.splitext(fname)[0] + ".jpg"
    out_path = os.path.join(folder, out_name)

    try:
        result = subprocess.run(
            ["sips", "-s", "format", "jpeg", in_path, "--out", out_path],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            # 🗑️ Удаляем исходный HEIC
            os.remove(in_path)
            converted.append(out_name)

            # === 🔄 Переименование JSON
            json_name = f"{os.path.splitext(fname)[0]}.HEIC.supplemental-metadata.json"
            new_json_name = f"{os.path.splitext(fname)[0]}.JPG.supplemental-metadata.json"

            json_path = os.path.join(folder, json_name)
            new_json_path = os.path.join(folder, new_json_name)

            if os.path.exists(json_path):
                # === 🔄 Переименование JSON
                os.rename(json_path, new_json_path)
                print(f"🔄 Переименован JSON: {json_name} → {new_json_name}")

                # === 📝 Исправляем "title" внутри JSON
                try:
                    with open(new_json_path, "r", encoding="utf-8") as file:
                        metadata = json.load(file)

                    # Обновляем поле "title"
                    metadata["title"] = out_name

                    # Сохраняем обратно
                    with open(new_json_path, "w", encoding="utf-8") as file:
                        json.dump(metadata, file, ensure_ascii=False, indent=4)

                    print(f"✅ Обновлен 'title' в JSON: {out_name}")
                except Exception as e:
                    print(f"❌ Ошибка при обновлении JSON {new_json_path}: {e}")
            else:
                print(f"⚠️ JSON не найден: {json_name}")
        else:
            failed.append((fname, result.stderr.strip()))
    except Exception as e:
        failed.append((fname, str(e)))

# === 📝 Результат
print(f"\n✅ Конвертировано и удалено: {len(converted)} HEIC → JPG")
if failed:
    print(f"❌ Ошибки: {len(failed)}")
    for fname, msg in failed:
        print(f" - {fname}: {msg}")

In [ ]:
# --- images>photos converter  - MAIN script --

In [ ]:
import os
from PIL import Image, ExifTags
import pillow_heif
import json
import re
import pandas as pd
from tqdm import tqdm
import glob
import math

# === 📁 Папки
ROOT = os.path.abspath(".")
SRC_FOLDER = os.path.join(ROOT, "images")
OUT_FOLDER = os.path.join(ROOT, "photos")
MISSING_FOLDER = os.path.join(ROOT, "missing_coords")
CSV_FOLDER = os.path.join(ROOT, "csv")
LOG_FILE = os.path.join(CSV_FOLDER, "import_log.txt")

os.makedirs(OUT_FOLDER, exist_ok=True)
os.makedirs(MISSING_FOLDER, exist_ok=True)
os.makedirs(CSV_FOLDER, exist_ok=True)

CSV_MAIN = os.path.join(CSV_FOLDER, "photos.csv")
CSV_MISSING = os.path.join(CSV_FOLDER, "missing_coords.csv")

# === EXIF utils
def convert_to_degrees(v):
    d, m, s = v
    return float(d) + float(m) / 60 + float(s) / 3600

def extract_gps(exif):
    gps_raw = {}
    for tag_id, val in exif.items():
        tag = ExifTags.TAGS.get(tag_id)
        if tag == "GPSInfo":
            for key in val:
                gps_tag = ExifTags.GPSTAGS.get(key)
                gps_raw[gps_tag] = val[key]
    if 'GPSLatitude' in gps_raw and 'GPSLongitude' in gps_raw:
        try:
            lat = convert_to_degrees(gps_raw['GPSLatitude'])
            if gps_raw.get('GPSLatitudeRef', 'N') != 'N':
                lat = -lat
            lon = convert_to_degrees(gps_raw['GPSLongitude'])
            if gps_raw.get('GPSLongitudeRef', 'E') != 'E':
                lon = -lon
            return lat, lon
        except:
            return None, None
    return None, None

def extract_date(exif):
    date_str = exif.get(36867) or exif.get(306)
    if date_str:
        match = re.match(r"(\d{4}):(\d{2}):(\d{2})", date_str)
        if match:
            return match.group(1), match.group(2), match.group(3)
    return None, None, None

Image.init()

def spiral_coords(lat, lon, index, step=0.0002):
    angle = index * (math.pi / 3)
    radius = step * (1 + index // 6)
    return lat + radius * math.cos(angle), lon + radius * math.sin(angle)

image_files = [f for f in os.listdir(SRC_FOLDER)
               if f.lower().endswith(('.jpg', '.jpeg', '.heic')) and "-edited" not in f.lower()]

main_records = []
missing_records = []
log_entries = []

for fname in tqdm(image_files, desc="📦 Обработка изображений", ncols=80):
    in_path = os.path.join(SRC_FOLDER, fname)
    ext = os.path.splitext(fname)[1].lower()

    try:
        if ext == ".heic":
            heif_file = pillow_heif.read_heif(in_path)
            image = Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw")
            exif = image.getexif()
            log_entries.append(f"[HEIC] Converted and loaded: {fname}")
        else:
            image = Image.open(in_path)
            exif = image._getexif() or {}
            log_entries.append(f"[IMG] Loaded: {fname}")

        lat, lon = extract_gps(exif)
        year, month, day = extract_date(exif)
        source_type = "EXIF"

        json_candidates = glob.glob(in_path + ".supplemental*.json")
        if not json_candidates:
            alt_path = in_path.replace(".jpg", ".JPG")
            json_candidates = glob.glob(alt_path + ".supplemental*.json")
        if not json_candidates:
            alt_path = in_path.replace(".jpeg", ".JPEG")
            json_candidates = glob.glob(alt_path + ".supplemental*.json")

        if json_candidates:
            try:
                with open(json_candidates[0], "r", encoding="utf-8") as jf:
                    meta = json.load(jf)
                if (lat is None or lon is None) and "geoData" in meta:
                    lat = meta["geoData"].get("latitude")
                    lon = meta["geoData"].get("longitude")
                    if lat and lon:
                        source_type = "JSON"
                        log_entries.append(f"[JSON] Used metadata for: {fname}")
            except Exception as e:
                log_entries.append(f"[ERROR] JSON read failed for {fname}: {e}")
                pass

        if not (year and month and day):
            year, month, day = "2099", "01", "01"

        if not re.search(r'IMG_\d{8}_', fname):
            base_name = f"IMG_{year}{month}{day}_{abs(hash(fname)) % 10**6}.jpg"
        else:
            base_name = fname.replace(".heic", ".jpg").replace(".HEIC", ".jpg")

        if lat and lon:
            out_path = os.path.join(OUT_FOLDER, base_name)
        else:
            out_path = os.path.join(MISSING_FOLDER, base_name)

        image.convert("RGB").save(out_path, "JPEG", quality=85)
        log_entries.append(f"[SAVE] {fname} → {out_path}")

        row = {
            'filename': base_name,
            'folder': os.path.basename(OUT_FOLDER if lat and lon else MISSING_FOLDER),
            'latitude': lat,
            'longitude': lon,
            'year': year,
            'month': month,
            'day': day,
            'source_path': fname,
            'source_type': source_type
        }

        if lat and lon:
            main_records.append(row)
        else:
            missing_records.append(row)
            log_entries.append(f"[MISSING] {fname} → No coordinates")

    except Exception as e:
        missing_records.append({
            'filename': fname,
            'folder': os.path.basename(MISSING_FOLDER),
            'latitude': None,
            'longitude': None,
            'year': None,
            'month': None,
            'day': None,
            'error': str(e),
            'source_path': fname,
            'source_type': None
        })
        log_entries.append(f"[ERROR] {fname}: {e}")

pd.DataFrame(main_records).to_csv(CSV_MAIN, index=False)
pd.DataFrame(missing_records).to_csv(CSV_MISSING, index=False)

print(f"\n✅ С координатами: {len(main_records)} → {CSV_MAIN}")
print(f"⚠️ Без координат : {len(missing_records)} → {CSV_MISSING}")
print(f"📁 Фото сохранены в: {OUT_FOLDER} и {MISSING_FOLDER}")
print(f"📝 Лог записан в: {LOG_FILE}")
print("🌍 Джиттер применён для повторяющихся координат")
print("\n✅ Завершено. Результаты в photos.csv и missing_coords.csv")


In [ ]:
# PROCESS

# -- Add coordinates to missing_coords.csv and rename it into manual_coords.csv to include
# remaining points
# 1. Rename missing_coords into manual_coords
# 2. Add coordinates (from manual_coords-Copy1.csv) or manually
# 3. Check folder (should be photos)
# 4. Run the script.
# 5. Check the last lines of photos.csv to see if file names correspond to the ones in missing_coords/
# 6. Move all files from missing_coords/ into photos/

In [ ]:
import pandas as pd
import os

CSV_MAIN = "csv/photos.csv"
CSV_EXTRA = "csv/manual_coords.csv"
CSV_INVALID = "csv/manual_invalid.csv"
LOG_FILE = "csv/merge_log.txt"

def log(message):
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(message + "\n")
    print(message)

# === Проверка наличия файлов
if not os.path.exists(CSV_MAIN):
    raise FileNotFoundError(f"Не найден основной файл: {CSV_MAIN}")
if not os.path.exists(CSV_EXTRA):
    raise FileNotFoundError(f"Не найден дополнительный файл: {CSV_EXTRA}")

# === Загрузка CSV
main_df = pd.read_csv(CSV_MAIN)
extra_df = pd.read_csv(CSV_EXTRA)

# === Проверка структуры
if set(main_df.columns) != set(extra_df.columns):
    raise ValueError("❌ Колонки не совпадают между CSV-файлами!")

# === Фильтрация координат
initial_len = len(extra_df)
invalid_rows = extra_df[
    extra_df["latitude"].isna() |
    extra_df["longitude"].isna() |
    (extra_df["latitude"] == 0.0) |
    (extra_df["longitude"] == 0.0) |
    (extra_df["latitude"] < -90) | (extra_df["latitude"] > 90) |
    (extra_df["longitude"] < -180) | (extra_df["longitude"] > 180)
]
valid_extra_df = extra_df.drop(invalid_rows.index)

# === Сохранение отклонённых
if not invalid_rows.empty:
    invalid_rows.to_csv(CSV_INVALID, index=False)

# === Лог
log("📋 ЛОГ ОБЪЕДИНЕНИЯ")
log(f"Исходных строк в manual_coords.csv: {initial_len}")
log(f"Допустимых строк: {len(valid_extra_df)}")
log(f"Отклонено строк: {len(invalid_rows)}")

# === Объединение
combined_df = pd.concat([main_df, valid_extra_df])
combined_df = combined_df.drop_duplicates(subset=["filename"])

# === Сохранение
combined_df.to_csv(CSV_MAIN, index=False)
log(f"✅ Объединение завершено. Финальное количество строк: {len(combined_df)}")

In [ ]:
# --- Script deletes all files in Google Drive/Photos
# --- Script uploads all files from photos/ to Google/Photos/

In [9]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from tqdm import tqdm
import os

# === Авторизация в Google Drive
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

# === ID целевой папки на Google Drive
FOLDER_ID = '1em81MElkxnaue5r92e9hPGezgOuw4nRL'
LOCAL_PHOTOS_FOLDER = 'photos/'

# === Очистка папки на Google Drive
print("🧹 Очистка папки на Google Drive...")
file_list = drive.ListFile({
    'q': f"'{FOLDER_ID}' in parents and trashed=false"
}).GetList()

for f in tqdm(file_list, desc="Удаление файлов"):
    f.Delete()

print("✅ Папка очищена.")

# === Загрузка новых файлов из локальной папки
print("⬆️  Загрузка новых файлов в Google Drive...")
local_files = [f for f in os.listdir(LOCAL_PHOTOS_FOLDER) if f.lower().endswith(('.jpg', '.jpeg'))]

for fname in tqdm(local_files, desc="Загрузка файлов"):
    file_path = os.path.join(LOCAL_PHOTOS_FOLDER, fname)
    gfile = drive.CreateFile({
        'title': fname,
        'parents': [{'id': FOLDER_ID}]
    })
    gfile.SetContentFile(file_path)
    gfile.Upload()

print("✅ Загрузка завершена.")


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=849244129200-hdfqohk1rs46hjekajgu7pa4jqrn9sqj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
🧹 Очистка папки на Google Drive...


Удаление файлов: 100%|██████████| 1460/1460 [13:38<00:00,  1.78it/s]


✅ Папка очищена.
⬆️  Загрузка новых файлов в Google Drive...


Загрузка файлов: 100%|██████████| 2431/2431 [3:10:57<00:00,  4.71s/it]  

✅ Загрузка завершена.


In [ ]:
# ---- Getting descriptions from Google photos via API

In [11]:
import csv
import os
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build_from_document
from tqdm import tqdm

# === Настройки ===
SCOPES = ['https://www.googleapis.com/auth/photoslibrary.readonly']
OUTPUT_CSV = 'csv/photos_descriptions.csv'
ALBUM_NAMES = ["PhotoMap 2022-2025", "PhotoMap 2019-2021", "PhotoMap 2017-2019", "PhotoMap 2014-2016", "PhotoMap 2010-2013", "PhotoMap 08-09"]

# === Аутентификация ===
flow = InstalledAppFlow.from_client_secrets_file(
    'client_secrets.json', SCOPES
)
credentials = flow.run_local_server(port=0)

# Чтение локального discovery файла
with open("photoslibrary_v1_discovery.json", "r") as f:
    discovery_doc = f.read()

# Создание сервиса
service = build_from_document(discovery_doc, credentials=credentials)

# === Поиск нужных альбомов ===
print("🔍 Поиск альбомов в Google Photos...")
albums = []
nextPageToken = None

while True:
    response = service.albums().list(pageSize=50, pageToken=nextPageToken).execute()
    albums.extend(response.get('albums', []))
    nextPageToken = response.get('nextPageToken')
    if not nextPageToken:
        break

target_albums = {album['title']: album['id'] for album in albums if album['title'] in ALBUM_NAMES}

# === Проверка найденных альбомов ===
if not target_albums:
    print("❌ Не найдены указанные альбомы в Google Photos")
else:
    print(f"✅ Найдены альбомы: {list(target_albums.keys())}")

# === Запись в CSV ===
os.makedirs('csv', exist_ok=True)
with open(OUTPUT_CSV, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['file_id', 'description'])

    for album_name, album_id in target_albums.items():
        print(f"\n📂 Альбом: {album_name}")
        nextPageToken = None
        
        with tqdm(desc=f"Загрузка из {album_name}", unit="фото", leave=True) as pbar:
            while True:
                results = service.mediaItems().search(
                    body={"albumId": album_id, "pageSize": 100, "pageToken": nextPageToken}
                ).execute()

                items = results.get('mediaItems', [])
                nextPageToken = results.get('nextPageToken')

                if not items:
                    break

                for item in items:
                    file_id = item.get('id', 'No ID')
                    description = item.get('description', 'No Description')
                    writer.writerow([file_id, description])

                # === Динамическое обновление прогресса ===
                pbar.update(len(items))

                if not nextPageToken:
                    break

    print(f"\n✅ Все данные успешно записаны в {OUTPUT_CSV}")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=849244129200-hdfqohk1rs46hjekajgu7pa4jqrn9sqj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60354%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fphotoslibrary.readonly&state=o3otxIUEcijTRIyPvoxZqfeQARifsC&access_type=offline
🔍 Поиск альбомов в Google Photos...
✅ Найдены альбомы: ['PhotoMap 2022-2025', 'PhotoMap 2019-2021', 'PhotoMap 2017-2019', 'PhotoMap 2014-2016', 'PhotoMap 2010-2013', 'PhotoMap 08-09']

📂 Альбом: PhotoMap 2022-2025


Загрузка из PhotoMap 2022-2025: 653фото [00:07, 87.67фото/s] 



📂 Альбом: PhotoMap 2019-2021


Загрузка из PhotoMap 2019-2021: 784фото [00:07, 101.69фото/s]



📂 Альбом: PhotoMap 2017-2019


...


📂 Альбом: PhotoMap 2010-2013


Загрузка из PhotoMap 2010-2013: 257фото [00:03, 83.82фото/s] 



📂 Альбом: PhotoMap 08-09


Загрузка из PhotoMap 08-09: 109фото [00:01, 72.67фото/s] 


✅ Все данные успешно записаны в csv/photos_descriptions.csv
